In [11]:
import random
import time
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

In [1]:
driver = webdriver.Chrome()
driver.get("https://www.linkedin.com/feed/")

In [2]:
CANDIDATE_HEADLINES = ['professor', 'hiring', 'recruiter', 'recruiting' ,'manager', 'sde manager', 'software engineering manager', 'principal software engineer', 'principal engineer', 'director', 'vice president']
connect_counter = 0
like_counter = 0

In [3]:
logs = []

In [4]:
def scrollDown(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [5]:
def hoverOver(header):
    hover = ActionChains(driver).move_to_element(header)
    hover.perform()

In [6]:
def checkIfRelevantCandidate(profile):
    intro = profile.get_attribute("textContent").lower().strip()
    logs.append(intro)
    for headline in CANDIDATE_HEADLINES:
        if headline in intro:
            return True
    return False

In [7]:
def sleepRandom(max_time):
    sleep_time = random.randint(1, max_time*1000)/1000.0
    time.sleep(sleep_time)

In [8]:
def connectWithPeople():
    profiles = driver.find_elements_by_xpath('//div[@class="feed-shared-actor__meta relative"]')
    for profile in profiles:
        try:
            if checkIfRelevantCandidate(profile):
                header = profile.find_element_by_xpath('.//span[@class="feed-shared-actor__name t-14 t-black t-bold hoverable-link-text"]')
                person_name = header.get_attribute("textContent").strip()
                hoverOver(header)
                sleepRandom(2)
                connect_btn = driver.find_element_by_xpath('//button[@data-control-name="hover_card_connect"]')
                if not connect_btn.is_enabled():
                    continue
                print("Sending a connect request to {0}".format(person_name))
                driver.execute_script("arguments[0].click();", connect_btn)
                connect_counter += 1
                sleepRandom(20)
        except Exception as ex:
            print(str(ex))

In [9]:
def writeLogsToFile():
    global logs
    try:
        with open('logs.txt', 'a') as writer:
            data = '\n'.join(logs).encode('utf-8')
            logs = []
            writer.write(data)
    except Exception as ex:
        print(str(ex))
        print('Unable to write logs to file')

In [ ]:
count = 0
while True:
    likes = driver.find_elements_by_xpath('//li-icon[@type="like-icon"]')
    for like in likes:
        if like.get_attribute('size') == 'small':
            continue
        driver.execute_script("arguments[0].click();", like)
        like_counter += 1
        print("Sleeping before next like")
        sleepRandom(20)
    print("Sleeping before scrolling")
    sleepRandom(100)
    connectWithPeople()
    scrollDown(driver)
    count += 1
    print("Already ran {0} iterations".format(count))
    print("Total likes made: {0}".format(like_counter))
    print("Total connects  : {0}".format(connect_counter))
    writeLogsToFile()
    if count % 200 == 0:
        driver.refresh()
        sleepRandom(10)
        

Sleeping before next like
Sleeping before scrolling
Sending a connect request to Iqra Dada
local variable 'connect_counter' referenced before assignment
Already ran 1 iterations
Total likes made: 2
Total connects  : 0
Sleeping before next like
Sleeping before scrolling
Already ran 2 iterations
Total likes made: 3
Total connects  : 0
Sleeping before scrolling
